In [5]:
import numpy as np

Creates bathymetry file based off of Becker(2014) Ipan, Guam site data 

In [ ]:
# Create SWASH .bot bathymetry files in /mnt/data for the user's 1D transect.
# - Reef flat: 0–400 m, depth = 0.5 m
# - Reef face: 400–580 m, slope = 0.06 (depth increases linearly to 11.3 m at 580 m)
# - Offshore buffer: 580–650 m, constant 11.3 m

import numpy as np
from pathlib import Path

out_dir = Path("Becker_2014/")
out_dir.mkdir(parents=True, exist_ok=True)

dx = 1.0
x_end = 650.0   # domain 0..650 m
nx = int(x_end / dx) + 1  # include endpoint
x = np.linspace(0.0, x_end, nx)

# Base depth profile
depth = np.zeros_like(x)

# Reef flat: 0–400 m at 0.5 m depth
depth[(x >= 0) & (x <= 400)] = 0.5

# Reef face: 400–580 m, slope 0.06
mask_slope = (x > 400) & (x <= 580)
depth[mask_slope] = 0.5 + 0.06 * (x[mask_slope] - 400.0)

# Offshore buffer: 580–650 m at 11.3 m depth
depth[x > 580] = 11.3

# Write helper to save a .bot file (two columns: x depth)
def write_bot(filepath: Path, xvals, zvals):
    header = "! x [m], depth [m, positive downward]\n"
    # Build 2-col array
    arr = np.column_stack([xvals, zvals])
    # Save with header; SWASH ignores lines starting with '!'
    with open(filepath, "w") as f:
        f.write(header)
        np.savetxt(f, arr, fmt=["%.2f", "%.4f"])

# export file
base_path = out_dir / "ipan_transect.bot"
write_bot(base_path, x, depth)


# Show a quick preview of the first 10 lines of the base file
preview_lines = 12
with open(base_path, "r") as f:
    preview = "".join([next(f) for _ in range(preview_lines)])

preview, str(base_path)


('! x [m], depth [m, positive downward]\n0.00 0.5000\n1.00 0.5000\n2.00 0.5000\n3.00 0.5000\n4.00 0.5000\n5.00 0.5000\n6.00 0.5000\n7.00 0.5000\n8.00 0.5000\n9.00 0.5000\n10.00 0.5000\n',
 'Becker_2014/ipan_transect.bot')

Create water level from tide fluctuations

In [9]:
# Create a uniform tide time series file for SWASH READinp WLEVel NONSTAT usage.
# Semidiurnal-like cycle over 12 hours, 10-min step. Mean 0.5 m, amplitude 0.35 m.
import numpy as np
from pathlib import Path

out = Path("Becker_2014/ipan/ipan_tide.wlv")

t_total_hr = 12.0
dt_min = 10.0
times_sec = np.arange(0, t_total_hr*3600 + 1e-6, dt_min*60)  # in seconds
mean = 0.5
amp = 0.35
eta = mean + amp * np.sin(2*np.pi*times_sec/(t_total_hr*3600))

with open(out, "w") as f:
    f.write("! time[s]  water_level[m]\n")
    for t, z in zip(times_sec, eta):
        f.write(f"{int(t):6d}  {z:8.4f}\n")

print("First 10 lines:")
print("\n".join(open(out).read().splitlines()[:11]))


First 10 lines:
! time[s]  water_level[m]
     0    0.5000
   600    0.5305
  1200    0.5608
  1800    0.5906
  2400    0.6197
  3000    0.6479
  3600    0.6750
  4200    0.7008
  4800    0.7250
  5400    0.7475
